In [ ]:
# Import packages
using CSV, DataFrames, Statistics, Random;

In [ ]:
# Set seed for reproducability
seed = 123;

In [ ]:
# Read dataset
df = CSV.read("data/input/imputed_diabetes_patients.csv", DataFrame);

In [ ]:
# Initialize results
results = DataFrame(lambda = [], base_score = [], base_score_actual = [], new_score = [])
train_scores_df = DataFrame(AUC = [], treatment = [], non_treatment = [])
test_scores_df = DataFrame(AUC = [], treatment = [], non_treatment = [])

# Define grid of lambdas
lambda_values = 0:0.2:1

for lambda in lambda_values

    println("Loop for lambda = $lambda")

    # Split data into train and test
    train_df = filter(row -> row.train_flag == 1, df);
    test_df = filter(row -> row.train_flag == 0, df);

    # Calculate outcome for given lambda   
    train_outcomes = lambda .* train_df.phys_health_days + (1-lambda) .* train_df.ment_health_days;
    test_outcomes = lambda .* test_df.phys_health_days + (1-lambda) .* test_df.ment_health_days;
    
    # Identify treatments
    train_treatments = train_df.diabetes_course;
    test_treatments = test_df.diabetes_course;
    
    # Remove features
    drop_vars = [
        "phys_health_status", "phys_health_days", "ment_health_status", "ment_health_days", "diabetes_course", 
        "depressed_household", "alcohol_household", "drugs_household", "prison_household", "sleep", "train_flag"
    ]
        
    train_X = select!(train_df, Not(drop_vars));
    test_X = select!(test_df, Not(drop_vars));

    # Define propensity model
    model_propensity = IAI.GridSearch(
        IAI.XGBoostClassifier(
            random_seed = seed,
            criterion = :entropy,
            max_categoric_levels_before_warning = 15
        ), 
        minbucket = [5, 10, 20, 50],
        max_depth = [3, 5, 7, 9], 
        num_estimators = [20, 50, 100, 200]
    );

    # Define outcome model
    model_outcome = IAI.GridSearch(
        IAI.XGBoostRegressor(
            random_seed = seed,
            criterion = :tweedie,
            max_categoric_levels_before_warning = 15
        ), 
        minbucket = [5, 10, 20, 50],
        max_depth = [3, 5, 7, 9], 
        num_estimators = [20, 50, 100, 200]
    );

    # Define reward learner
    reward_lnr = IAI.CategoricalRegressionRewardEstimator(
        propensity_estimator = model_propensity,
        outcome_estimator = model_outcome,
        reward_estimator = :doubly_robust,
        random_seed = seed,
        propensity_insample_num_folds = 5, 
        outcome_insample_num_folds = 5,
        propensity_min_value = 0.001
    );

    # Train reward learner
    train_predictions, train_reward_score = IAI.fit_predict!(
        reward_lnr, train_X, train_treatments, train_outcomes,
        propensity_score_criterion = :auc, 
        outcome_score_criterion = :tweedie
    );
    train_rewards = train_predictions[:reward];

    # Define optimal policy tree
    grid = IAI.GridSearch(
        IAI.OptimalTreePolicyMinimizer(
            random_seed = seed,
            max_categoric_levels_before_warning = 20,
        ),
        max_depth = 2:6,
        minbucket = [5, 10, 20, 50]
    )

    # Train optimal policy tree
    IAI.fit!(grid, train_X, train_rewards)

    IAI.write_html("data/output/tree_lambda=$lambda", grid)
    IAI.write_questionnaire("data/output/questions_lambda=$lambda", grid)

    # Predict outcomes on test set
    test_predictions, test_reward_score = IAI.fit_predict!(
        reward_lnr, test_X, test_treatments, test_outcomes,
        propensity_score_criterion = :auc, 
        outcome_score_criterion = :tweedie
    )
    test_rewards = test_predictions[:reward]

    # Calculate outcomes for baseline and new model
    base_score = mean([test_rewards[i, test_treatments[i]] for i in 1:length(test_treatments)])
    base_score_actual = mean(test_outcomes)

    pred_treatments = IAI.predict(grid, test_X)
    pred_outcomes = [test_rewards[i, pred_treatments[i]] for i in 1:length(pred_treatments)]
    new_score = mean(pred_outcomes)

    new_row = (lambda = lambda, base_score = base_score, base_score_actual = base_score_actual, new_score = new_score)
    push!(results, new_row)
    CSV.write("data/output/results_lambda.csv", results)

    # Calculate performance metrics
    new_row_train = (AUC = train_reward_score[:propensity], treatment = train_reward_score[:outcome]["yes"], non_treatment = train_reward_score[:outcome]["no"])
    new_row_test = (AUC = test_reward_score[:propensity], treatment = test_reward_score[:outcome]["yes"], non_treatment = test_reward_score[:outcome]["no"])
    push!(train_scores_df, new_row_train)
    push!(test_scores_df, new_row_test)
    CSV.write("data/output/train_scores_df.csv", train_scores_df)
    CSV.write("data/output/test_scores_df.csv", test_scores_df)
end